### Submission notebook

In [ ]:
!cp /kaggle/input/run-1280-yolov5l/best.pt /kaggle/working/best.pt
!cp /kaggle/input/run-1280-yolov5l/yolov_train3_results/last.pt /kaggle/working/yolov5l_train3_last.pt
# !cp /kaggle/input/run-1280-yolov5l/yolov5l6_train3_weights/last.pt /kaggle/working/yolov5l6_last.pt
# !cp /kaggle/input/run-1280-yolov5l/yolov5l6_train5_weights/last.pt /kaggle/working/yolov5l6_last.pt
!cp /kaggle/input/run-1280-yolov5l/yolov5l6_train6_weights/last.pt /kaggle/working/yolov5l6_last.pt
# !cp /kaggle/input/run-1280-yolov5l/yolov5s6_train1_weights/best.pt /kaggle/working/yolov5s6_best.pt
# !cp /kaggle/input/reef-yolor-w6/yoloR_w6_train/train1/best_r.pt /kaggle/working/yolor_w6.pt

In [ ]:
# setup - copy font for offline use
!mkdir -p /root/.config/Ultralytics
!cp /kaggle/input/yolov5-font/Arial.ttf /root/.config/Ultralytics/

In [ ]:
# !pip install /kaggle/input/pycocotools/pycocotools-2.0.3.tar

In [ ]:
# %cd /kaggle/working/
# !rm -rf yolor
# %cp -r ../input/hey-yolor/. /kaggle/working/yolor

### Imports

In [ ]:
# %cd /kaggle/working/yolor
# from utils.torch_utils import select_device
# import pycocotools
# # YoloR methods
# from yolor.models.models import Darknet
# from yolor.utils.datasets import letterbox
# from yolor.utils.general import non_max_suppression, scale_coords

In [ ]:
%cd /kaggle/working

In [ ]:
import numpy as np
import torch
import cv2
import matplotlib.pyplot as plt
import matplotlib 
%matplotlib inline
import pandas as pd
from PIL import Image
import os
from pathlib import Path
import sys
import gc
import json5
import shutil

sys.path.append('../input/tensorflow-great-barrier-reef')
sys.path.append('../input/heyyolov5')
sys.path.append('../input/yolov5pip/yolov5-pip')

sys.path.append('../input/weightedboxesfusion')
from ensemble_boxes import *

sys.path.append('../input/isears-tf-clahe')
import tf_clahe

np.random.seed(0)

import tensorflow as tf
tf.version

torch.no_grad()

### Helper Funcs

In [ ]:
# Memory saving function credit to https://www.kaggle.com/gemartin/load-data-reduce-memory-usage
def reduce_mem_usage(df):
    """ iterate through all the columns of a dataframe and modify the data type to reduce memory usage.        
    """
    start_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage of dataframe is {:.2f} MB'.format(start_mem))

    for col in df.columns:
        col_type = df[col].dtype

        if col_type != object:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)

    end_mem = df.memory_usage().sum() / 1024**2
    print('Memory usage after optimization is: {:.2f} MB'.format(end_mem))
    print('Decreased by {:.1f}%'.format(100 * (start_mem - end_mem) / start_mem))
    del start_mem, end_mem
    return df

def load_labels():
    labels = pd.read_csv("/kaggle/input/tensorflow-great-barrier-reef/train.csv", skipinitialspace=True)
    labels.drop_duplicates(inplace=True)
    labels.drop(['sequence'], axis = 1, inplace=True)
    labels.drop(['sequence_frame'], axis = 1, inplace=True)
    labels = reduce_mem_usage(labels)
    return labels

def get_path(row):
    row['image_path'] = f'/kaggle/input/tensorflow-great-barrier-reef/train_images/video_{row.video_id}/{row.video_frame}.jpg'
    return row

def process_annot(ann_data):
    yolo_labels = []
    metric_boxes = []
    if ann_data and ann_data != '[]':
        ann_splits = json5.loads(ann_data)
        for ann_split in ann_splits:
            row = dict()
            # load annotation data as json and get values
            a_data = json5.loads(ann_split)
            row['xmin'] = int(a_data["x"])
            row['ymin'] = int(a_data["y"])
            row['xmax'] = int(a_data["x"]) + int(a_data["width"])
            row['ymax'] = int(a_data["y"]) + int(a_data["height"])
            metric_boxes.append(np.array([row['xmin'],row['ymin'],a_data["width"],a_data["height"]]))
            row['score'] = 1.0
            row['label'] = 'starfish'
            yolo_labels.append(row)
        del ann_splits
    return yolo_labels, metric_boxes
 
def get_sample_imgs(sample_count = 50):
    labels = load_labels()
    labels = labels[labels['annotations'].str.len() > 49]
    labels = labels.apply(get_path, axis=1)
    return labels.sample(sample_count)

### Image Processing Helpers

In [ ]:
def plot_images(img1, img2, img3, title='', lbl1='', lbl2='', lbl3 = ''):
    plt.close('all')
    fig, ax = plt.subplots(1, 3, figsize=(23,13))
    ax[0].imshow(img1, cmap = plt.get_cmap(name = 'gray'))
    ax[0].set_axis_off()
    ax[0].set_title(lbl1, fontsize=18)
    ax[1].imshow(img2, cmap = plt.get_cmap(name = 'gray'))
    ax[1].set_axis_off()
    ax[1].set_title(lbl2, fontsize=18)
    ax[2].imshow(img3, cmap = plt.get_cmap(name = 'gray'))
    ax[2].set_axis_off()
    ax[2].set_title(lbl3, fontsize=18)
    fig.suptitle(title, fontsize=22, y=0.81)
    plt.tight_layout()
    plt.show()

@tf.function(experimental_compile=True)  # Enable XLA
def tf_clahe_gpu(og_img):
    # channel 1 is grayscale and 3 is RGB
    # og_img = tf.io.decode_jpeg(tf.io.read_file(img_path), channels=3)
    correct_img = tf_clahe.clahe(og_img, tile_grid_size=(28, 28), clip_limit=3.3, gpu_optimized=True)
    return correct_img, og_img

def tf_enhance_image_gpu(og_img):
    import torch
    torch.cuda.empty_cache()
    corrected_img, og_img = tf_clahe_gpu(tf.constant(og_img))
    n_img = tf.image.adjust_gamma(corrected_img, 1.2)
    return n_img.numpy()

# further read for good tips/tricks
# https://www.kaggle.com/soumya9977/learning-to-sea-underwater-img-enhancement-eda#%F0%9F%8E%AF-Main-Working-Code

# CLAHE (Contrast Limited Adaptive Histogram Equalization)
def clhae(img):
    import cv2
    clahe = cv2.createCLAHE(clipLimit=3.3, tileGridSize=(27,27))
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)  # convert from BGR to LAB color space
    l, a, b = cv2.split(lab)  # split on 3 different channels
    l2 = clahe.apply(l)  # apply CLAHE to the L-channel
    lab = cv2.merge((l2,a,b))  # merge channels
    img = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)  # convert from LAB to BGR
    del lab, l2, l, clahe
    return img

# auto white balance
# credit to https://gist.github.com/DavidYKay/9dad6c4ab0d8d7dbf3dc
def white_balance(img, percent=1):
    out_channels = []
    cumstops = (
        img.shape[0] * img.shape[1] * percent / 200.0,
        img.shape[0] * img.shape[1] * (1 - percent / 200.0)
    )
    for channel in cv2.split(img):
        cumhist = np.cumsum(cv2.calcHist([channel], [0], None, [256], (0,256)))
        low_cut, high_cut = np.searchsorted(cumhist, cumstops)
        lut = np.concatenate((
            np.zeros(low_cut),
            np.around(np.linspace(0, 255, high_cut - low_cut + 1)),
            255 * np.ones(255 - high_cut)
        ))
        out_channels.append(cv2.LUT(channel, lut.astype('uint8')))
        del cumhist, low_cut, high_cut
    del cumstops
    return cv2.merge(out_channels)

# auto gamma correction
# why 1.2? because, I felt it is better visually
def gamma_correction(img, gamma =1.2):
    igamma = 1.0 / gamma
    imin, imax = img.min(), img.max()
    img_c = img.copy()
    img_c = ((img_c - imin) / (imax - imin)) ** igamma
    img_c = img_c * (imax - imin) + imin
    del imin, imax, igamma
    return img_c.astype(np.uint8)

# combined helper function
# NOTE : ORDER MATTERS HERE
def enhance_image(img):
    # why in this order? again, visually it gives a lot better results
    # also, photographer recommend to gamma correct -> white balance -> color correction
    _img = img.copy()
    _img = gamma_correction(_img)
    _img = white_balance(_img)
    _img = clhae(_img)
    return _img

def convert_to_gray(img):
    return cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

def invert_image(img):
    # invert
    return cv2.bitwise_not(img.copy())

def drawBoundingBoxes(image_data, infer_results):
    image_data = cv2.cvtColor(image_data, cv2.COLOR_BGR2RGB)
    pinkish_red = (227, 27, 90)
    for res in infer_results:
        left = int(res['xmin'])
        top = int(res['ymin'])
        right = int(res['xmax'])
        bottom = int(res['ymax'])
        score = res['score']
        label = res['label'] if res['label'] is not None else 'starfish'
        im_h, im_w, _ = image_data.shape
        thick = int((im_h + im_w) // 750) * 2
        # draw bb
        cv2.rectangle(image_data,(left, top), (right, bottom), pinkish_red, thick)
        tf = max(thick - 1, 1)   # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=thick / 3, thickness=int(tf))[0]
        c2 = left + t_size[0], top - t_size[1] + 2
        # draw text fill
        cv2.rectangle(image_data, (left, top ), c2, pinkish_red, -1, cv2.LINE_AA)  # filled
        # draw text
        cv2.putText(image_data, f'{label} {score}', (left, top - 6), 0, 1e-3 * im_h, (255,255,255), int(thick*0.5))
        del left, top, right, bottom, score, label, im_h, im_w, tf, t_size, c2
        
    return image_data

def draw_fused_boxes(image, boxes_list, scores_list, labels_list, image_size=(1280, 720)):
    thickness = 5
#     image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    thick = int((image_size[0] + image_size[1]) // 750) * 2
    for i in range(len(boxes_list)):
        x1 = int(image_size[0] * boxes_list[i][0])
        y1 = int(image_size[1] * boxes_list[i][1])
        x2 = int(image_size[0] * boxes_list[i][2])
        y2 = int(image_size[1] * boxes_list[i][3])
        lbl = labels_list[i]  
        score = round(scores_list[i], 2)
        label = 'starfish'
        tf = max(thick - 1, 1)   # font thickness
        t_size = cv2.getTextSize(label, 0, fontScale=thick / 3, thickness=int(tf))[0]
        c2 = x1 + t_size[0], y1 - t_size[1] + 2
        cv2.rectangle(image, (x1, y1), (x2, y2), (227, 27, 90), thick, cv2.LINE_AA)
        # draw text fill
        cv2.rectangle(image, (x1, y1), c2, (227, 27, 90), -1, cv2.LINE_AA)  # filled
        # draw text
        cv2.putText(image, f'{label} {score}', (x1, y1 - 6), 0, 1e-3 * image_size[1], (255,255,255), int(thick*0.5))
    return image

# best score : 502 with iou_thr=0.45 & skip_box_thr=0.15
# best score : 513 with iou_thr=0.51 & skip_box_thr=0.11
def fuse_boxes(img, boxes_list, scores_list, labels_list, draw_boxes = True,
               gen_submit_str = True, iou_thr=0.51, skip_box_thr=0.11):
    if not boxes_list or len(boxes_list) <=0:
        return dict(
        img= img,
        boxes= (),
        scores= (), 
        labels= (), 
        detect_count= 0,
        submission_str= '')
#     weights = [2, 1, 3] # weights/priority to model data
    # skip_box_thr : We skip boxes with confidence lower than skip_box_thr
    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=None, iou_thr=iou_thr, skip_box_thr=skip_box_thr)

    ann = ' '
    im_h, im_w = img.shape[:2]
    metric_boxes = []
    if gen_submit_str and boxes is not None and len(boxes) > 0:
        
        score_sum = np.sum(scores)
        pred_count = len(boxes)
        if pred_count >= 1:
            avg_conf = score_sum/pred_count
            print(f'[Fusion] Total starfishs : {pred_count}, Average confidence : {avg_conf}')
        else:
            print("[Fusion] No starfish detected")
            
        for idx, bb in enumerate(boxes):
            xmin = bb[0]*im_w
            ymin = bb[1]*im_h
            xmax = bb[2]*im_w
            ymax = bb[3]*im_h
            w = xmax - xmin
            h = ymax - ymin
            score = round(scores[idx], 2)
            ann += f'{score} {int(xmin)} {int(ymin)} {int(w)} {int(h)}'
            metric_boxes.append(np.array([score,xmin,ymin,w,h]))
            ann +=' '
        print(f'fuse {ann}')

        if draw_boxes:
            img = draw_fused_boxes(img, boxes, scores, labels)

    return dict(
        img= img,
        boxes = boxes,
        scores= scores, 
        labels= labels,
        metric_boxes=metric_boxes,
        detect_count = len(boxes),
        submission_str= ann.strip(' '))

def free():
    import gc
    import torch
    import matplotlib.pyplot as plt
    gc.collect()
    if torch.cuda.is_available():
        torch.cuda.empty_cache() 
        
def tf_save_image(img, fname):
    path = f'/kaggle/working/{fname}.jpg'
    enc = tf.image.encode_jpeg(img)
    tf.io.write_file(tf.constant(path), enc)
    return path

### Model Helpers

In [ ]:
# load model from yolov5
def load_model(model_path = "/kaggle/working/best.pt", conf = 0.40, iou = 0.5):
    from yolov5 import YOLOv5

    if not torch.cuda.is_available():
        print('using cpu')
        device = 'cpu'
    else:
        device = 'cuda:0'
        print('using gpu')
        
    yolov5 = YOLOv5(model_path, device)
    print('conf : ', conf, ' iou : ', iou)
    yolov5.model.conf = conf
    yolov5.model.iou = iou
    yolov5.model.max_det = 1000
    return yolov5

# best score 0.502 with m1_conf=0.39, m1_iou=0.51, m2_conf=0.09, m2_iou=0.69, m3_conf=0.21, m3_iou=0.79
# new best score 0.513 with m1_conf=0.10, m1_iou=0.51, m2_conf=0.09, m2_iou=0.41, m3_conf=0.10, m3_iou=0.51
def load_ensemble(m1_conf=0.10, m1_iou=0.51, m2_conf=0.09, m2_iou=0.41, m3_conf=0.10, m3_iou=0.51, m4_conf = 0.35, m4_iou=0.51):
    with torch.no_grad():
        m1 = load_model(conf=m1_conf, iou=m1_iou)
        m2 = load_model('/kaggle/working/yolov5l_train3_last.pt', conf=m2_conf, iou=m2_iou)
        m3 = load_model('/kaggle/working/yolov5l6_last.pt', conf=m3_conf, iou=m3_iou)

        return m1, m2, m3

# process prediction results
def process_preds(results, im_w = 1280, im_h=720, model_id = ''):
    default_res = dict(
        boxes= (),
        scores= (), 
        labels= (), 
        detect_count= 0)
    
    res = results.pandas().xyxy
    if not res:
        return default_res
    rp1 = res[0]
    if rp1 is None or rp1.empty:
        return default_res

    del res
    # normalize values for ensemble box func
    rp1['xmin_normalized'] = rp1['xmin'] / im_w
    rp1['ymin_normalized'] = rp1['ymin'] / im_h
    rp1['xmax_normalized'] = rp1['xmax'] / im_w
    rp1['ymax_normalized'] = rp1['ymax'] / im_h
    rp1['w'] = rp1['xmax'] - rp1['xmin']
    rp1['h'] = rp1['ymax'] - rp1['ymin']
    bboxes  = rp1[['xmin_normalized','ymin_normalized','xmax_normalized','ymax_normalized']].values.astype(float)
    scores = rp1.confidence.values
    labels = rp1['class'].values
    
    idxs = np.where(scores >= 0.12)[0]
    bboxes = bboxes[idxs]
    scores = scores[idxs]
    labels = labels[idxs]
    
    score_sum = np.sum(scores)
    pred_count = len(scores)
    if pred_count >= 1:
        avg_conf = score_sum/pred_count
        print(f'[{model_id}] Total starfishs : {pred_count}, Average confidence : {avg_conf}')
    else:
        print(f'[{model_id}] No starfish detected')
    
    return dict(
        boxes= bboxes,
        scores= scores, 
        labels= labels, 
        detect_count= pred_count)

# predict helper
def predict(model, img, imgsz = 1280, draw_boxes = True, model_id = '', augment = True):
    with torch.no_grad():
        # yolo-pip package has "predict" function
        # ultralytics/yolo has constructor predict
        predict = getattr(model, "predict", None)

        if callable(predict):
            results = model.predict(img, size=imgsz, augment=augment)
        else:
            results = model(img, size=imgsz, augment=augment)

        proc_res = process_preds(results, 1280, 720, model_id=model_id)
        del results
        if draw_boxes:
            img = draw_fused_boxes(img, proc_res['boxes'], proc_res['scores'], proc_res['labels'])

        proc_res['img'] = img
        return proc_res

def submission_predict(model1, model2, model3, img, submit_mode = False, fuse_iou_thr=0.51, fuse_skip_box_thr=0.10):
    free()
    draw_boxes = not submit_mode
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

#     enhanced_image = enhance_image(img)

    enhanced_image = tf_enhance_image_gpu(img)
    
    m1_res = predict(model1, img, imgsz=1280, draw_boxes = draw_boxes, model_id = 'Ensemble M1 TTA-False', augment=False)
    m2_res = predict(model2, enhanced_image, imgsz=1280, draw_boxes = draw_boxes, model_id = 'Ensemble M2 TTA-False', augment=False)
    m3_res = predict(model3, enhanced_image, imgsz=2048, draw_boxes = draw_boxes, model_id = 'Ensemble M3 TTA-False', augment=False)

    del enhanced_image
    free()

    fusion_res = fuse_boxes(m2_res['img'], \
                [m1_res['boxes'],m2_res['boxes'],m3_res['boxes']], \
                [m1_res['scores'],m2_res['scores'],m3_res['scores']], \
                [m1_res['labels'],m2_res['labels'],m3_res['labels']], \
                            draw_boxes = draw_boxes, iou_thr=fuse_iou_thr, skip_box_thr=fuse_skip_box_thr)

    if submit_mode:
        return fusion_res['submission_str']
    
    del m1_res, m2_res, m3_res, img
    return  fusion_res

def submission_predict_perf(model1, model2, model3, img, fuse_iou_thr=0.51, fuse_skip_box_thr=0.10):
    return submission_predict(model1, model2, model3, img, submit_mode = True, fuse_iou_thr=fuse_iou_thr, fuse_skip_box_thr=fuse_skip_box_thr)

In [ ]:
def calc_iou(bboxes1, bboxes2, bbox_mode='xywh'):
    assert len(bboxes1.shape) == 2 and bboxes1.shape[1] == 4
    assert len(bboxes2.shape) == 2 and bboxes2.shape[1] == 4
    
    bboxes1 = bboxes1.copy()
    bboxes2 = bboxes2.copy()
    
    if bbox_mode == 'xywh':
        bboxes1[:, 2:] += bboxes1[:, :2]
        bboxes2[:, 2:] += bboxes2[:, :2]

    x11, y11, x12, y12 = np.split(bboxes1, 4, axis=1)
    x21, y21, x22, y22 = np.split(bboxes2, 4, axis=1)
    xA = np.maximum(x11, np.transpose(x21))
    yA = np.maximum(y11, np.transpose(y21))
    xB = np.minimum(x12, np.transpose(x22))
    yB = np.minimum(y12, np.transpose(y22))
    interArea = np.maximum((xB - xA + 1), 0) * np.maximum((yB - yA + 1), 0)
    boxAArea = (x12 - x11 + 1) * (y12 - y11 + 1)
    boxBArea = (x22 - x21 + 1) * (y22 - y21 + 1)
    iou = interArea / (boxAArea + np.transpose(boxBArea) - interArea)
    return iou

def f_beta(tp, fp, fn, beta=2):
    return (1+beta**2)*tp / ((1+beta**2)*tp + beta**2*fn+fp)

def calc_is_correct_at_iou_th(gt_bboxes, pred_bboxes, iou_th, verbose=False):
    gt_bboxes = gt_bboxes.copy()
    pred_bboxes = pred_bboxes.copy()
    
    tp = 0
    fp = 0
    for k, pred_bbox in enumerate(pred_bboxes): # fixed in ver.7
        ious = calc_iou(gt_bboxes, pred_bbox[None, 1:])
        max_iou = ious.max()
        if max_iou > iou_th:
            tp += 1
            gt_bboxes = np.delete(gt_bboxes, ious.argmax(), axis=0)
        else:
            fp += 1
        if len(gt_bboxes) == 0:
            fp += len(pred_bboxes) - (k + 1) # fix in ver.7
            break

    fn = len(gt_bboxes)
    return tp, fp, fn

def calc_is_correct(gt_bboxes, pred_bboxes, iou_th=0.5):
    """
    gt_bboxes: (N, 4) np.array in xywh format
    pred_bboxes: (N, 5) np.array in conf+xywh format
    """
    if len(gt_bboxes) == 0 and len(pred_bboxes) == 0:
        tps, fps, fns = 0, 0, 0
        return tps, fps, fns

    elif len(gt_bboxes) == 0:
        tps, fps, fns = 0, len(pred_bboxes)*11, 0
        return tps, fps, fns

    elif len(pred_bboxes) == 0:
        tps, fps, fns = 0, 0, len(gt_bboxes)*11
        return tps, fps, fns

    pred_bboxes = pred_bboxes[pred_bboxes[:,0].argsort()[::-1]] # sort by conf

    tps, fps, fns = 0, 0, 0
    tp, fp, fn = calc_is_correct_at_iou_th(gt_bboxes, pred_bboxes, iou_th)
    tps += tp
    fps += fp
    fns += fn
    return tps, fps, fns

def calc_f2_score(gt_bboxes_list, pred_bboxes_list, verbose=False):
    """
    gt_bboxes_list: list of (N, 4) np.array in xywh format
    pred_bboxes_list: list of (N, 5) np.array in conf+xywh format
    """
    f2s = []
    for iou_th in np.arange(0.3, 0.85, 0.05):
        tps, fps, fns = 0, 0, 0
        for gt_bboxes, pred_bboxes in zip(gt_bboxes_list, pred_bboxes_list):
            tp, fp, fn = calc_is_correct(gt_bboxes, pred_bboxes, iou_th)
            tps += tp
            fps += fp
            fns += fn
            if verbose:
                num_gt = len(gt_bboxes)
                num_pred = len(pred_bboxes)
                print(f'num_gt:{num_gt:<3} num_pred:{num_pred:<3} tp:{tp:<3} fp:{fp:<3} fn:{fn:<3}')
        f2 = f_beta(tps, fps, fns, beta=2)    
        print(f'f2@ iou: {round(iou_th,2)}, f2 : {round(f2,2)}, f2(with comp difference) : {round(f2-0.18, 2)}')
        f2s.append(f2)
        
    f2mean = np.mean(f2s)
    print(f'f2 mean is {f2mean}, with competition difference {f2mean-0.178}')
    return f2mean

In [ ]:
# def convert_to_cxywh(bboxes):
#     predictions = []
#     detects = []
    
#     for i in range(len(bboxes)):
#         box = bboxes[i]
#         score = round(scores[i],2)
#         x_min = int(box[0])
#         y_min = int(box[1])
#         x_max = int(box[2])
#         y_max = int(box[3])
        
#         bbox_width = x_max - x_min
#         bbox_height = y_max - y_min
#         detects.append([score, x_min, y_min, bbox_width, bbox_height])
#         predictions.append('{:.2f} {} {} {} {}'.format(score, x_min, y_min, bbox_width, bbox_height))
#         return detects, ' '.join(predictions)
    
# def show_prediction(img, bboxes, scores):

#     for box, score in zip(bboxes, scores):
#         cv2.rectangle(img, (int(box[0]), int(box[1])), (int(box[2]), int(box[3])), (0,0,255), 2)
#         cv2.putText(img, f'{score:.2f}', (int(box[0]), int(box[1]-3)), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0,0,255), 1, cv2.LINE_AA)
    
#     img = img[:,:,::-1]
#     img = Image.fromarray(img).resize((1280, 720))
#     return img

# def load_yolor_model(model_path, cfg_path, img_size, device):
#     with torch.no_grad():
#         model = Darknet(cfg_path).cuda()
#         model.load_state_dict(torch.load(model_path, map_location=device)['model'])
#         model.to(device).eval()
#         model.half()

#         init = torch.zeros((1, 3, img_size, img_size), device=device)
#         _ = model(init.half()) if device.type != 'cpu' else None 
#         return model

# def predict_img(model, img_in, img_size, conf_thres, iou_thres, device):
#     with torch.no_grad():
#         bboxes = []
#         scores = []
#         classes = []

#         img = letterbox(img_in, new_shape=img_size, auto_size=64)[0]
#         img = np.ascontiguousarray(img[:, :, ::-1].transpose(2, 0, 1))
#         img = (torch.from_numpy(img).to(device).half() / 255.0).unsqueeze(0)

#         pred = model(img)[0]

#         if pred is not None:
#             pred = non_max_suppression(pred, conf_thres=conf_thres, iou_thres=iou_thres, merge=False, classes=None, agnostic=False)

#         for i, det in enumerate(pred):
#             if det is not None and len(det):
#                 det[:, :4] = scale_coords(img.shape[2:], det[:, :4], img_in.shape).round()    

#         det = det.cpu().detach().numpy()

#         bboxes = det[:, :4] 
#         scores = det[:, 4] 
#         classes = det[:, 5]
        
#         norm_boxes = []
#         for i in range(len(bboxes)):
#             box = bboxes[i]
#             x_min = round(box[0]/img_in.shape[1],4)
#             y_min = round(box[1]/img_in.shape[0],4)
#             x_max = round(box[2]/img_in.shape[1],4)
#             y_max = round(box[3]/img_in.shape[0],4)
#             norm_boxes.append([x_min, y_min, x_max, y_max])
        
#         return bboxes, scores, classes, norm_boxes

In [ ]:
# device = select_device('0')
# model = load_yolor_model('./yolor_w6.pt', './yolor/cfg/yolor_w6.cfg', 6400, device)

In [ ]:
# img = cv2.imread('../input/tensorflow-great-barrier-reef/train_images/video_0/9651.jpg')
# # img = cv2.imread('../input/tensorflow-great-barrier-reef/train_images/video_0/9700.jpg')
# # # img = cv2.imread('../input/tensorflow-great-barrier-reef/train_images/video_0/9674.jpg')
# img = enhance_image(img)
# bboxes, scores, classes, norm_boxes = predict_img(model, img, 6400, conf_thres = 0.01, iou_thres = 0.3, device = device)
# display(show_prediction(img, bboxes, scores))

In [ ]:
# def timereps(func, reps = 1):
#     from time import time
#     start = time()
#     for i in range(0, reps):
#         func()
#     end = time()
#     return (end - start) / reps

In [ ]:
# del model
# free()
# model = load_model('/kaggle/working/yolov5l6_last.pt', conf=0.15, iou=0.51)

In [ ]:
# img = cv2.imread('../input/tensorflow-great-barrier-reef/train_images/video_2/5803.jpg')
# # # # img = cv2.imread('../input/tensorflow-great-barrier-reef/train_images/video_0/19.jpg')
# # # # img = cv2.imread('../input/tensorflow-great-barrier-reef/train_images/video_0/12.jpg')
# # # free()
# # # # model1, model2, model3 = load_ensemble()
# # # # def infer():
# # # #     print(submission_predict_perf(model1, model2, model3, img))
    
# # # # # timereps(infer)
# # # # # infer()
# # # # del model1, model2, model3
# # # # print('before ', img)
# # img = enhance_image(img)
# img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
# pred = predict(model, img, 2048)
# fusion_res = fuse_boxes(img,[pred['boxes']], [pred['scores']], [pred['labels']], draw_boxes = True, iou_thr=0.3, skip_box_thr=0.05)
# display(Image.fromarray(fusion_res['img']))
# # # cv2.normalize(img, img, 0, 255, cv2.NORM_MINMAX)

# # # # print('after ', img)
# # # pred = predict(model, img, enhance_img=True)
# # # fusion_res = fuse_boxes(img,[pred['boxes']], [pred['scores']], [pred['labels']], draw_boxes = True)
# # # # display(Image.fromarray(fusion_res['img']))

# # # # fusion_res['submission_str']
# # # # print('res ', fusion_res['submission_str'])
# # # del model
# # # free()

### Run infer on train samples

In [ ]:
import sys
sys.path.append('../input/weightedboxesfusion')
from ensemble_boxes import *

def show_original(img, annotations):
    og_bb_list, m_boxes = process_annot(annotations)
    return drawBoundingBoxes(img, og_bb_list), og_bb_list

def infer_samples(benchmark_mode = False):
    from tqdm import tqdm
    # calculate the f2-measure
    from sklearn.metrics import fbeta_score
    from sklearn.metrics import f1_score
    from sklearn.metrics import precision_score
    from sklearn.metrics import recall_score
    import matplotlib.pyplot as plt
    %matplotlib inline
    free()

    os.chdir('/kaggle/working/')

    m1_sc = 0.0
    m2_sc = 0.0
    labels = get_sample_imgs(1000)[:40]
    print(len(labels.values))
    
#     model = load_model('/kaggle/working/yolov5l6_last.pt', conf=0.05, iou=0.3)
#     model2 = load_model('/kaggle/working/yolov5l6_last.pt', conf=0.15, iou=0.45)
#     model1 = load_model('/kaggle/working/best.pt', conf=0.12, iou=0.3)

#     model1, model2, model3 = load_ensemble(m1_conf=0.15, m1_iou=0.51, m2_conf=0.09, m2_iou=0.51, m3_conf=0.19, m3_iou=0.51)
#     model1, model2, model3 = load_ensemble(m1_conf=0.32, m1_iou=0.51, m2_conf=0.09, m2_iou=0.51, m3_conf=0.19, m3_iou=0.51)
#     model1, model2, model3 = load_ensemble(m1_conf=0.12, m1_iou=0.51, m2_conf=0.09, m2_iou=0.45, m3_conf=0.18, m3_iou=0.51)

    # F2 scores over 3 iterations are : [0.7, 0.6500000000000001, 0.7]
    model1, model2, model3 = load_ensemble(m1_conf=0.05, m1_iou=0.30, m2_conf=0.09, m2_iou=0.40, m3_conf=0.09, m3_iou=0.40)
#     model3 = load_model('/kaggle/working/yolov5l6_last.pt', conf=0.12, iou=0.50)
    if not benchmark_mode:
        g_ytrue = []
        m1_ypred = []
        m2_ypred = []
    
    gt_preds = []
    x_preds = []
    for i, la in tqdm(labels.iterrows(), total=len(labels.values)):
        path = la['image_path']
        annotations = la['annotations']
        if os.path.exists(path) and os.path.isfile(path):
            img = cv2.imread(path)
#             img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

            if benchmark_mode:
                print(submission_predict_perf(model1, model2, model3, img))
            else:
                img1, og_bb_list = show_original(img, annotations)

                # 0.497 score with 0.55, 0.40
                # 0.520 score with fuse_iou_thr=0.50, fuse_skip_box_thr=0.19 [0.725, 0.625, 0.725]
                # with iou=0.55 & skip_conf=0.40, f2 over 3 iter [0.775, 0.775, 0.8] (2nd run [0.825, 0.925, 0.775])
                # with 0.45, 0.35 [0.7, 0.825, 0.675]
                # with 0.45, 0.40 [0.8,0.8,0.7],[0.775, 0.675, 0.650]
                fusion_res = submission_predict(model1, model2, model3, img, fuse_iou_thr=0.65, fuse_skip_box_thr=0.09)

                og_bb_list, m_boxes = process_annot(annotations)
                
                for x in m_boxes:
                    gt_preds.append(x)
                    
                for x in fusion_res['metric_boxes']:
                    x_preds.append(x)
                
                g_ytrue.append(len(og_bb_list))
                print('Ground Truth ', len(og_bb_list))
#                 m1_ypred.append(m1_res['detect_count'])
#                 m1_ypred.append(len(norm_boxes))
#                 m1_ypred.append(detect_count)
                m2_ypred.append(fusion_res['detect_count'])
#                 display(Image.fromarray(fusion_res['img']))
#                 plot_images(img1, m1_res['img'], fusion_res['img'], 'Image Comparison',
#                             f'Ground Truth \n{path}', 'Single Model', 'Ensembled Results')
                del img, img1, fusion_res, og_bb_list
    
                plt.show()
                
        del path, annotations

    free()
    if not benchmark_mode:
        gt_preds = np.array(gt_preds)
        x_preds = np.array(x_preds)
        
        print('G1 ytrue', g_ytrue)
#         print('m1 ypred', m1_ypred)
        print('m2 ypred', m2_ypred)

#         m1_sc = fbeta_score(g_ytrue, m1_ypred, average='micro', beta=2.0)
        m2_sc = fbeta_score(g_ytrue, m2_ypred, average='micro', beta=2.0)
#         print('F2 Beta Sore for Model 1 : ', m1_sc)
        print('F2 Beta Score for Ensemble: ', m2_sc)
#         del g_ytrue, m1_ypred, m2_ypred
        calc_f2_score([gt_preds], [x_preds], verbose=False)
    
#     del labels, model1, model2, model3
    return m2_sc

# timereps(infer_samples)
# infer_samples()

# n_iter = 3
# f2_scores = []

# for i in range(n_iter):
#     f2_scores.append(infer_samples())
    
# print(f'F2 scores over {n_iter} iterations are : {f2_scores}')

### Submission

In [ ]:
# free()

In [ ]:
import greatbarrierreef

def create_submission(disable_make_env = False):
    from IPython.display import display
    os.chdir('/kaggle/working/')
    free()

#     model1, model2, model3 = load_ensemble(m1_conf=0.32, m1_iou=0.51, m2_conf=0.09, m2_iou=0.51, m3_conf=0.19, m3_iou=0.51)
#     model1, model2, model3 = load_ensemble(m1_conf=0.09, m1_iou=0.51, m2_conf=0.09, m2_iou=0.51, m3_conf=0.12, m3_iou=0.51)
    
#     model1, model2, model3 = load_ensemble(m1_conf=0.05, m1_iou=0.3, m2_conf=0.05, m2_iou=0.3, m3_conf=0.05, m3_iou=0.3)
    
#     model = load_model('/kaggle/working/yolov5l6_last.pt', conf=0.05, iou=0.3)
#     model2 = load_model('/kaggle/working/yolov5l6_last.pt', conf=0.15, iou=0.45)
#     model1 = load_model('/kaggle/working/best.pt', conf=0.12, iou=0.3)

    model1, model2, model3 = load_ensemble(m1_conf=0.05, m1_iou=0.30, m2_conf=0.09, m2_iou=0.40, m3_conf=0.09, m3_iou=0.40)

    if not disable_make_env:
        env = greatbarrierreef.make_env() 
        
    iter_test = env.iter_test()
    for img, pred_df in iter_test:
        free()
        
        fusion_res = submission_predict(model1, model2, model3, img, fuse_iou_thr=0.65, fuse_skip_box_thr=0.09)

        print('predicted annotation ',  fusion_res['submission_str'])
        pred_df['annotations'] = fusion_res['submission_str']
        
        env.predict(pred_df)
        free()
    
#     del model1, model2, model3
    gc.collect()
    
# run create_submission
create_submission()

In [ ]:
pd.read_csv('submission.csv')

In [ ]:
# cleanup
!rm /kaggle/working/best.pt
!rm /kaggle/working/yolov5l_train3_last.pt
!rm /kaggle/working/yolov5l6_last.pt
!rm /kaggle/working/yolov5s6_best.pt